In [1]:
import os
import re
import MeCab
import pickle
import gensim
import ast
import lda
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
import gensim.corpora as corpora
from gensim import similarities 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import spacy
from pprint import pprint
import warnings
import logging
from google.cloud import storage as gcs
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from scipy.spatial.distance import pdist 
from scipy.spatial.distance import squareform
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
from collections import Counter
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
%matplotlib inline

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/protobuf/descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from google.protobuf.pyext import _message


In [2]:
INPUT_BUCKET = 'gs://mj-labeling-questionnaires-dev.datasets.linkbal.com/inputs/ver_01/'
PROJECT_NAME = 'linkbal-dp'
BUCKET_NAME = 'mj-labeling-questionnaires-dev.datasets.linkbal.com' 
client = gcs.Client(PROJECT_NAME)
bucket = gcs.Bucket(client, name=BUCKET_NAME)
jp_stop_words = open("/Users/jesse.ulundo/demo_tests/Japanese_stopword_list.txt", "r")
qs = pd.read_csv(INPUT_BUCKET + 'mj_questionnaire_text.csv', sep=',')

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/



## Removing Noise From Data

In [3]:
qs = qs.drop(['want_recommend', 'want_recommend_reason'], axis=1)
qs = qs.dropna()
qs = qs.drop_duplicates() 
qs

,id,overall_satisfaction,overall_satisfaction_reason
4,111376,6.0,とても楽しかったです\nいろんな人と友達になれたのが◎
5,111377,2.0,店舗スタッフの対応が悪い
7,111379,2.0,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...
8,111380,4.0,結果が伴ってなかったから
9,111381,2.0,想像していた人数よりも少ない。
...,...,...,...
20305,131673,3.0,相手の人が少なくて暇な時間がさみしかったです。
20306,131674,3.0,街コンには問題はなかったけど、いい出会いはありませんでした
20307,131675,1.0,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...
20308,131676,2.0,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。


In [4]:
qs = qs[qs.overall_satisfaction < 3]
qs = qs[qs.overall_satisfaction_reason != '上記同様']
qs = qs[qs.overall_satisfaction_reason != '同上']
qs

,id,overall_satisfaction,overall_satisfaction_reason
5,111377,2.0,店舗スタッフの対応が悪い
7,111379,2.0,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...
8,111380,4.0,結果が伴ってなかったから
9,111381,2.0,想像していた人数よりも少ない。
11,111383,2.0,大人数なのにマンツーマン、フリータイムが少ない。
...,...,...,...
20305,131673,3.0,相手の人が少なくて暇な時間がさみしかったです。
20306,131674,3.0,街コンには問題はなかったけど、いい出会いはありませんでした
20307,131675,1.0,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...
20308,131676,2.0,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。


## Tokenizing and Stemming

In [5]:
tagger = MeCab.Tagger()
def tokenize_stemmer(text):
    lemmas = []
    for line in tagger.parse(text).splitlines()[:-1]:
        surface, feature = line.split('\t')
        if feature.split(',')[6] != '*':
            lemmas.append(feature.split(',')[6])
    return lemmas

In [6]:
categories = ['参加人数に関して', '男女比率に関して', 'スタッフ対応に関して', '飲食内容に関して', '虚偽記載']
categories_tkn = [tokenize_stemmer(text) for text in categories]
categories_tkn

[['参加', '人数', 'に関して'],
 ['男女', '比率', 'に関して'],
 ['スタッフ', '対応', 'に関して'],
 ['飲食', '内容', 'に関して'],
 ['虚偽', '記載']]

In [7]:
qs['tokenized_overall_satisfaction_reason'] = qs.apply(lambda row: tokenize_stemmer(row['overall_satisfaction_reason']), axis=1)
qs

,id,overall_satisfaction,overall_satisfaction_reason,tokenized_overall_satisfaction_reason
5,111377,2.0,店舗スタッフの対応が悪い,"[店舗, スタッフ, の, 対応, が, 悪い]"
7,111379,2.0,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...,"[他, の, の, お, 店, だ, と, 店員, や, 常連, さん, が, 気, を, ..."
8,111380,4.0,結果が伴ってなかったから,"[結果, が, 伴う, てる, ない, た, から]"
9,111381,2.0,想像していた人数よりも少ない。,"[想像, する, て, いる, た, 人数, より, も, 少ない, 。]"
11,111383,2.0,大人数なのにマンツーマン、フリータイムが少ない。,"[大, 人数, だ, のに, マンツーマン, 、, フリー, タイム, が, 少ない, 。]"
...,...,...,...,...
20305,131673,3.0,相手の人が少なくて暇な時間がさみしかったです。,"[相手, の, 人, が, 少ない, て, 暇, だ, 時間, が, さみしい, た, です..."
20306,131674,3.0,街コンには問題はなかったけど、いい出会いはありませんでした,"[街, コン, に, は, 問題, は, ない, た, けど, 、, いい, 出会い, は,..."
20307,131675,1.0,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...,"[最終, 的, に, 社長, から, 返金, 対応, する, 旨, の, 提案, を, 受け..."
20308,131676,2.0,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。,"[水上, バス, は, 夜景, が, きれい, だ, た, けど, 、, ほんとに, それ,..."


In [8]:
stop = [words.strip("\n") for words in jp_stop_words]
qs['tokenized_overall_satisfaction_reason'] = qs['tokenized_overall_satisfaction_reason'].apply(lambda x: [item for item in x if item not in stop])
qs

,id,overall_satisfaction,overall_satisfaction_reason,tokenized_overall_satisfaction_reason
5,111377,2.0,店舗スタッフの対応が悪い,"[店舗, スタッフ, 対応, 悪い]"
7,111379,2.0,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...,"[店員, 常連, きかせる, 比べる, どうしても, 多い]"
8,111380,4.0,結果が伴ってなかったから,"[結果, 伴う]"
9,111381,2.0,想像していた人数よりも少ない。,"[想像, 人数, 少ない]"
11,111383,2.0,大人数なのにマンツーマン、フリータイムが少ない。,"[大, 人数, マンツーマン, フリー, タイム, 少ない]"
...,...,...,...,...
20305,131673,3.0,相手の人が少なくて暇な時間がさみしかったです。,"[相手, 少ない, 暇, さみしい]"
20306,131674,3.0,街コンには問題はなかったけど、いい出会いはありませんでした,"[街, コン, 問題, けど, 出会い]"
20307,131675,1.0,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...,"[最終, 社長, 返金, 対応, 旨, 提案, 受ける, 案内, 約束, 未だ, 待つ, 状..."
20308,131676,2.0,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。,"[水上, バス, 夜景, きれい, けど, ほんとに, だけ, 違和感, すぎる]"


## Generating Bigrams and Corpus

In [9]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [10]:
def get_corpus(df):
    #df['text'] = strip_newline(df.text)
    words = stop
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

In [11]:
qs_new = qs[pd.notnull(qs['tokenized_overall_satisfaction_reason'])]
qs_new['tokenized_overall_satisfaction_reason'] = [' '.join(map(str, l)) for l in qs['tokenized_overall_satisfaction_reason']]
qs_new
print(qs_new['tokenized_overall_satisfaction_reason'])

5                                            店舗 スタッフ 対応 悪い
7                                  店員 常連 きかせる 比べる どうしても 多い
8                                                    結果 伴う
9                                                想像 人数 少ない
11                                 大 人数 マンツーマン フリー タイム 少ない
                               ...                        
20305                                        相手 少ない 暇 さみしい
20306                                       街 コン 問題 けど 出会い
20307    最終 社長 返金 対応 旨 提案 受ける 案内 約束 未だ 待つ 状況 本当に 返金 もらえ...
20308                      水上 バス 夜景 きれい けど ほんとに だけ 違和感 すぎる
20311                                   参加 費 交通 費 無駄 やる 詐欺
Name: tokenized_overall_satisfaction_reason, Length: 7299, dtype: object


In [12]:
data_satis = qs_new.tokenized_overall_satisfaction_reason.values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data_satis))
data_words[:5]

[['店舗', 'スタッフ', '対応', '悪い'],
 ['店員', '常連', 'きかせる', '比へる', 'とうしても', '多い'],
 ['結果', '伴う'],
 ['想像', '人数', '少ない'],
 ['人数', 'マンツーマン', 'フリー', 'タイム', '少ない']]

In [13]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
print(trigram_mod[bigram_mod[data_words[0]]])

['店舗', 'スタッフ', '対応', '悪い']


In [14]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [15]:
data_words_bigrams = make_bigrams(data_words)
nlp = spacy.load('ja_ginza', disable=['parser', 'ner'])
nlp

In [16]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(10, 1), (11, 1)],
 [(12, 1), (13, 1), (14, 1)],
 [(12, 1), (13, 1), (15, 1), (16, 1)],
 [(17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 2),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1)],
 [(44, 1)],
 [(45, 1), (46, 1), (47, 1), (48, 1)],
 [(49, 1), (50, 1), (51, 1), (52, 1), (53, 1)],
 [(6, 1),
  (12, 1),
  (26, 2),
  (31, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 2),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 3),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 2),
  (86, 1)],
 [(31, 1),
  (42, 1)

In [17]:
size_id2word= [[(id2word[id], freq) for id, freq in cp] for cp in corpus]

In [18]:
print("Vocabulary size: {}".format(len(id2word)))
print("Corpus size: {}".format(len(corpus)))

Vocabulary size: 5297
Corpus size: 7299


## Create LDA Dissatisfaction_Reason TM

In [19]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                           id2word=id2word, 
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model

In [20]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.151*"参加" + 0.144*"人数" + 0.109*"少ない" + 0.081*"男性" + 0.066*"女性" + '
  '0.044*"料理" + 0.026*"開催" + 0.016*"へし" + 0.015*"中止" + 0.014*"過きる"'),
 (1,
  '0.048*"テーフル" + 0.043*"年齢" + 0.032*"食へる" + 0.031*"高い" + 0.031*"トリンク" + '
  '0.028*"最初" + 0.026*"こ飯" + 0.022*"値段" + 0.016*"美味しい" + 0.014*"聞こえる"'),
 (2,
  '0.043*"来る" + 0.038*"楽しい" + 0.035*"主催" + 0.030*"通り" + 0.025*"クルーフ" + '
  '0.020*"二度と" + 0.020*"途中" + 0.019*"帰る" + 0.015*"出す" + 0.014*"もらえる"'),
 (3,
  '0.088*"男女" + 0.034*"違う" + 0.032*"記載" + 0.027*"全く" + 0.027*"替え" + 0.026*"比率" '
  '+ 0.025*"やる" + 0.024*"レヘル" + 0.021*"詐欺" + 0.019*"ひとい"'),
 (4,
  '0.059*"対応" + 0.049*"出会い" + 0.048*"という" + 0.035*"内容" + 0.016*"事前" + '
  '0.015*"聞く" + 0.014*"すこい" + 0.014*"当日" + 0.012*"のみ" + 0.011*"飲食"'),
 (5,
  '0.053*"食事" + 0.047*"出来る" + 0.037*"せる" + 0.028*"いたたく" + 0.022*"無駄" + '
  '0.020*"盛り上かる" + 0.018*"ほほ" + 0.016*"すっと" + 0.016*"いたたける" + 0.016*"よく"'),
 (6,
  '0.033*"スタッフ" + 0.031*"すきる" + 0.027*"イヘント" + 0.020*"多い" + 0.020*"話す" + '
  '0.015*"会場" + 0.014*"た

In [21]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.286797885632482

Coherence Score:  0.3428142092036123


In [22]:
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#vis

## Inferring The Categories From The Topics

In [23]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words):
    sent_topics_df = pd.DataFrame()
    
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_contribution', 'Topic_Keywords']
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [24]:
categories_df = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words)
df_dominant_categories = categories_df.reset_index()
df_dominant_categories.columns = ['Document_No', 'Dominant_Categories', 'Category_perc_contrib', 'Keywords', 'Overall_Satisfaction_Reason']
df_dominant_categories

,Document_No,Dominant_Categories,Category_perc_contrib,Keywords,Overall_Satisfaction_Reason
0,0,6.0,0.4319,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[店舗, スタッフ, 対応, 悪い]"
1,1,6.0,0.4799,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[店員, 常連, きかせる, 比へる, とうしても, 多い]"
2,2,6.0,0.3740,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[結果, 伴う]"
3,3,6.0,0.3559,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[想像, 人数, 少ない]"
4,4,6.0,0.3502,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[人数, マンツーマン, フリー, タイム, 少ない]"
...,...,...,...,...,...
7294,7294,6.0,0.4044,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[相手, 少ない, さみしい]"
7295,7295,6.0,0.4318,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[コン, 問題, けと, 出会い]"
7296,7296,6.0,0.4317,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[最終, 社長, 返金, 対応, 提案, 受ける, 案内, 約束, 未た, 待つ, 状況, ..."
7297,7297,6.0,0.4536,"スタッフ, すきる, イヘント, 多い, 話す, 会場, たけ, 連絡, 行く, また","[水上, ハス, 夜景, きれい, けと, ほんとに, たけ, 違和感, すきる]"


In [30]:
df_dominant_categories.Dominant_Categories.unique()
df_dominant_categories.to_csv(r'/Users/jesse.ulundo/demo_tests/ov_sat_TM.csv', index = None, header=True)

In [26]:
#def qs_categories(model_type=lda_model, corpus=corpus, texts=data_words):
    #qs_cat_df = pd.DataFrame()
    #for i, row in enumerate(model_type[corpus]):
        #row = sorted(row[0], key= lambda x: (x[0]), reverse=False)
        #print(row)
    #topics_percentage_df = pd.DataFrame(row)
    #return topics_percentage_df
        #for j, (topic_num, prop_topic)in enumerate(row):
            #print(j, topic_num, prop_topic)
            #if j == 0:
                #wp = model_type.show_topic(topic_num)
                #topic_keywords = " ".join([word for word, prop in wp])
                #print(topic_keywords)
                #qs_cat_df = qs_cat_df.append(pd.Series([int(topic_num), round(prop_topic,2), round(prop_topic,2), round(prop_topic,2),round(prop_topic,2), round(prop_topic,2), topic_keywords]), ignore_index=True)
            #else:
                #break
    #qs_cat_df.columns = ['Dominant_Topic', 'Perc_Dom_Topic_0', 'Perc_Dom_Topic_1', 'Perc_Dom_Topic_2', 'Perc_Dom_Topic_3', 'Perc_Dom_Topic_4', 'Topic_Keywords']
    #contents = pd.Series(texts)
    #qs_cat_df = pd.concat([qs_cat_df, contents], axis=1)
    #return(qs_cat_df)

In [27]:
#qs_categories(model_type=lda_model, corpus=corpus, texts=data_words)
#categories_df = qs_categories(model_type=lda_model, corpus=corpus, texts=data_words)
#categories_df
#questionnaire_cat = categories_df.reset_index()
#questionnaire_cat.columns = ['Document_No', 'Dominant_Topic', 'Perc_Topic_0', 'Perc_Topic_1', 'Perc_Topic_2', 'Perc_Topic_3', 'Perc_Topic_4', 'Dom_Top_Keywords', 'Overall_Satisfaction_Reason']
#questionnaire_cat

In [28]:
#questionnaire_cat.head(50)

In [29]:
#[スタッフ対応, 会場と, 雰囲気, 参加者の年齢, 少数の参加者, 女性と男性の出会い, 異性と話すのが難しい, other]